In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv('Power_Data.csv')
df.head()

,Material,Customer,Movement type,Posting Date,MBG Quantity,SM3 Quantity,Qty (MMSCMD),Customer Name,Customer Location
0,GMS_GGS1_4,10333.0,601.0,1-Apr-20,"-25,343.609","-679,414.000",0.679,NTPC LTD,BHARUCH_N.T.P.C LIMITED(10333 NGSGU
1,GMS_GGS1_4,10333.0,601.0,1-Apr-20,"-4,827.422","-129,414.000",0.129,NTPC LTD,BHARUCH_N.T.P.C LIMITED(10333 NGSGU
2,GMS_GGS1_4,10333.0,602.0,1-Apr-20,"4,827.422","129,414.000",0.129,NTPC LTD,BHARUCH_N.T.P.C LIMITED(10333 NGSGU
3,GMS_NCG,10333.0,601.0,1-Apr-20,-448.409,"-12,021.000",0.012,NTPC LTD,BHARUCH_N.T.P.C LIMITED(10333 NGSGU
4,GMS_NCG,10333.0,601.0,1-Apr-20,-448.409,"-12,021.000",0.012,NTPC LTD,BHARUCH_N.T.P.C LIMITED(10333 NGSGU


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35392 entries, 0 to 35391
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Material           35391 non-null  object 
 1   Customer           35391 non-null  float64
 2   Movement type      35391 non-null  float64
 3   Posting Date       35391 non-null  object 
 4   MBG Quantity       35391 non-null  object 
 5   SM3 Quantity       35391 non-null  object 
 6   Qty (MMSCMD)       35391 non-null  float64
 7   Customer Name      35391 non-null  object 
 8   Customer Location  35391 non-null  object 
dtypes: float64(3), object(6)
memory usage: 2.4+ MB


In [14]:
df_copy = df.copy()
df.drop(['Material', 'Movement type', 'MBG Quantity', 'SM3 Quantity','Customer Name'], axis=1, inplace=True)

In [17]:
df_copy.columns

Index(['Material', 'Customer', 'Movement type', 'Posting Date', 'MBG Quantity',
       'SM3 Quantity', 'Qty (MMSCMD)', 'Customer Name', 'Customer Location'],
      dtype='object')

In [19]:
id_mapping = df_copy.groupby('Customer').apply(lambda x:x[['Customer Name','Customer Location']].to_dict('records')).to_dict
print(id_mapping)


C:\Users\rachitgandhi2\AppData\Local\Temp\ipykernel_17212\2205674619.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  id_mapping = df_copy.groupby('Customer').apply(lambda x:x[['Customer Name','Customer Location']].to_dict('records')).to_dict


<bound method Series.to_dict of Customer
10054.0    [{'Customer Name': 'Caparo Power Private Limit...
10114.0    [{'Customer Name': 'PRAGATI POWER CORP. LTD.',...
10150.0    [{'Customer Name': 'Tamil Nadu Generation & Di...
10151.0    [{'Customer Name': 'Arkay Energy (Rameswaram) ...
10226.0    [{'Customer Name': 'INDRAPRASTH POWER GEN. COM...
10282.0    [{'Customer Name': 'Maharashtra State Power Ge...
10307.0    [{'Customer Name': 'Caparo Power Private Limit...
10331.0    [{'Customer Name': 'NTPC Ltd', 'Customer Locat...
10333.0    [{'Customer Name': 'NTPC LTD', 'Customer Locat...
10335.0    [{'Customer Name': 'N.T.P.C LTD.', 'Customer L...
10336.0    [{'Customer Name': 'NTPC LTD', 'Customer Locat...
10337.0    [{'Customer Name': 'NTPC LTD', 'Customer Locat...
10347.0    [{'Customer Name': 'MMS STEEL & POWER PVT LTD'...
10353.0    [{'Customer Name': 'Caparo Power Private Limit...
10371.0    [{'Customer Name': 'Shell Energy India Private...
10377.0    [{'Customer Name': 'PRAGATI POWER

In [28]:
def search_id(id_to_search,id_mapping):
    if id_to_search in id_mapping:
        results = id_mapping[id_to_search]
        if results.len == 1:
            print("One entry found:\n")
            print("ID: {id_to_search}:\t Name: {results[0]['Customer Name']}\\t Address: {results[0]['Customer Location']}")
        else:
            print('Multiple or No Entries Found')

In [33]:
search_id(10333,id_mapping)

TypeError: argument of type 'method' is not iterable